In [2]:
import json
import pandas as pd
from pathlib import Path

file_path_data = Path('/Users/adrianfranz/Documents/Studium/6. Semester/Unternehmenssoftware/Projekt/data/player_data_prepared2000-2023.json')
    
with open(file_path_data) as file:
    data = json.load(file)

def build_rows(player):
    player_id = player['playerID']
    name = player['name']
    age = player['age']
    
    seasons = pd.DataFrame(player['seasons'])
    
    # Normalizing cards column
    cards = seasons.pop('cards').apply(pd.Series)
    seasons = pd.concat([seasons, cards], axis=1)
    
    # Convert season to int and create cl column
    seasons['season'] = seasons['season'].apply(convert_season)
    seasons['cl'] = seasons['league'].apply(lambda x: 1 if x == 'Champions League' else 0)
    
    # Aggregate data per season
    combined_seasons = seasons.groupby('season').agg({
        'games': 'sum',
        'goals': 'sum',
        'assists': 'sum',
        'yellow': 'sum',
        'yellowred': 'sum',
        'red': 'sum',
        'time_played': 'sum',
        'cl': 'sum'
    }).reset_index()
    
    combined_seasons['age'] = age
    combined_seasons['id'] = player_id
    combined_seasons['name'] = name
    
    return combined_seasons


def convert_season(season):
    if '/' in season:
        return int(season.split('/')[0]) + 2000
    return int(season)


def build_df(data):
    player_dfs = [build_rows(player) for player in data if player['seasons']]
    return pd.concat(player_dfs)


In [68]:
df = build_df(data)

In [109]:
df = df[df['season'] != 2023]
df = df[df['season'] != 2022]
df = df[df['season'] != 2019]
df = df.reset_index(drop=True)



In [69]:
file_path_winner = Path('/Users/adrianfranz/Documents/Studium/6. Semester/Unternehmenssoftware/Projekt/data/winner.csv')

df_winner = pd.read_csv(file_path_winner)

In [70]:
df_winner['season'] = df_winner['year'] -1
df_winner = df_winner.fillna('')

top_players_by_season = {}
for index, row in df_winner.iterrows():
    season = row['season']
    if season not in top_players_by_season:
        top_players_by_season[season] = []
    top_players_by_season[season].extend([row['#1'], row['#2'], row['#3']])

In [110]:
df['top_player'] = 0

for index, row in df.iterrows():
    season = row['season']
    player_name = row['name']
    if season in top_players_by_season and player_name in top_players_by_season[season]:
        df.at[index, 'top_player'] = 1

In [111]:
mask = df['name'] == 'Karim Benzema'
rows_with_karim = df[mask]


In [115]:
df['goals_per_game'] = df['goals'] / df['games']
df['assists_per_game'] = df['assists'] / df['games']
df['cards_per_game'] = df['yellow'] + df['yellowred'] + df['red'] / df['games']

In [117]:
df.head(100)

,season,games,goals,assists,yellow,yellowred,red,time_played,cl,age,id,name,top_player,goals_per_game,cards_per_game,assists_per_game
0,2000,29,2,5,3,0,0,709,0,55,3,Andreas Buck,0,0.068966,3.0,0.172414
1,2001,3,0,0,0,0,0,58,0,55,3,Andreas Buck,0,0.000000,0.0,0.000000
2,2002,12,2,1,2,0,0,518,0,55,3,Andreas Buck,0,0.166667,2.0,0.083333
3,2000,35,5,7,5,0,0,803,0,55,4,Youri Djorkaeff,0,0.142857,5.0,0.200000
4,2001,16,4,0,0,0,0,1141,0,55,4,Youri Djorkaeff,0,0.250000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2004,42,6,4,2,0,0,361,0,45,19,Marco Reich,0,0.142857,2.0,0.095238
96,2005,26,4,0,2,0,0,236,0,45,19,Marco Reich,0,0.153846,2.0,0.000000
97,2006,16,0,0,1,0,0,793,0,45,19,Marco Reich,0,0.000000,1.0,0.000000
98,2007,9,0,0,1,0,0,421,0,45,19,Marco Reich,0,0.000000,1.0,0.000000


In [ ]:
df.to_csv('dataset.csv', index=False)